In [89]:
import json
import matplotlib.pyplot as plt
import pandas as pd


# Load times from CVC4 or timeout
def load_mirabelle_data(filename):
    with open(filename, 'r') as f:
        data = json.load(f)

    times = {}
    for theorem, entry in data.items():
        if entry['method'] == 'cvc5':
            times[theorem] = entry['total_time'] / 1000 # time is in ms
        else:
            times[theorem] = None
    return times

# Load times from E-Graph file
def load_egraph_data(filename):
    with open(filename, 'r') as f:
        data = json.load(f)

    return {theorem: float(entry['summary']['total_time']) for theorem, entry in data.items() if entry['summary']['stop_reason'] == {"Other":"Found equivalence"}}

data = {'lemma':{}, 'no_lemma': {}, 'egraph': {}}

for src in ['alive', 'alive_bitwise']:
    base = f'./data/{src}'
    data['lemma'].update(load_mirabelle_data(base + '/lemma/parsed.json'))
    data['no_lemma'].update(load_mirabelle_data(base + '/no_lemma/parsed.json'))
    data['egraph'].update(load_egraph_data(base + '/egraph_stats.json'))



df = pd.DataFrame(data)
df.sort_values(by='egraph', ascending=False)

,lemma,no_lemma,egraph
muldivrem_239,71.813,NaN,0.007288
AddSub_1614,38.942,278.047,0.004483
AddSub_1564,NaN,NaN,0.001942
AndOrXor_2475,NaN,NaN,0.001897
InstCombineShift497d,NaN,NaN,0.000869
...,...,...,...
AndOrXor_2515,NaN,290.134,NaN
AndOrXor_1294,NaN,NaN,NaN
AndOrXor_2581,NaN,NaN,NaN
AndOrXor_2265,NaN,NaN,NaN


In [90]:
import os

indices = df.index.to_list()

artifact_dir = '../alive_bench/processed_artifact'

for file in os.listdir(artifact_dir):
    name = file.removesuffix('.json')
    print(file, name)
    with open(os.path.join(artifact_dir,file), 'r') as f:
        data = json.load(f)
    
    data = {k.replace(':', '_').replace('-', '_'):v for (k,v) in data.items()}
    
    # using combined since it was the most effective method out of all of them
    new_data = {k: data[k]['combined']['time'] for k in indices if data[k]['combined']['status'] == 'unsat'}
    df[name] = new_data

df

cvc4_tplanes.json cvc4_tplanes
cvc4_tplanes_saturate_no_e_matching.json cvc4_tplanes_saturate_no_e_matching
vampire.json vampire
z3_default.json z3_default


,lemma,no_lemma,egraph,cvc4_tplanes,cvc4_tplanes_saturate_no_e_matching,vampire,z3_default
AddSub_1614,38.942,278.047,0.004483,0.92,14.29,NaN,NaN
muldivrem_152,26.983,NaN,0.000155,0.90,NaN,NaN,NaN
Select_1105,1.294,1.186,0.000001,0.73,0.03,0.44,0.28
InstCombineShift497d,NaN,NaN,0.000869,NaN,NaN,NaN,NaN
AddSub_1164,27.148,279.151,0.000309,0.63,2.85,49.13,6.42
...,...,...,...,...,...,...,...
AndOrXor_2515,NaN,290.134,NaN,NaN,NaN,NaN,NaN
AndOrXor_1294,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AndOrXor_2581,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AndOrXor_2265,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
import pandas as pd
import re
from collections import defaultdict

def extract_prefix(idx):
    m = re.match(r"([a-zA-Z]+)(.*)", idx)
    return m.group(1) if m else idx

def latex_escape(text):
    if isinstance(text, str):
        return text.replace("_", r"\_")
    else:
        return str(text)

groups = defaultdict(list)
for idx in df.index:
    groups[extract_prefix(idx)].append(idx)

combined_rows = []

for prefix, indices in groups.items():
    sub_df = df.loc[indices]
    solved_mask = sub_df.notna().any(axis=1)
    solved_df = sub_df[solved_mask].copy()

    solved_df['solvers_count'] = solved_df.notna().sum(axis=1)

    sort_cols = ['solvers_count']
    if 'egraph' in solved_df.columns:
        sort_cols.append('egraph')
    solved_df = solved_df.sort_values(by=sort_cols)

    # For terminal print: show header, solved_df.to_string(), and unsolved list
    print(f"\n=== Problem Class: {prefix} ===")
    if solved_df.empty:
        print("No solved problems.")
    else:
        print(solved_df.to_string())

    unsolved = sub_df[~solved_mask].index.tolist()
    if unsolved:
        print("Unsolved problems:", unsolved)

    # For LaTeX table building, append rows like before
    combined_rows.append(("__GROUP_HEADER__", prefix))
    for idx, row in solved_df.iterrows():
        combined_rows.append((idx, row))
    if unsolved:
        combined_rows.append(("__UNSOLVED__", unsolved))

columns = list(df.columns) + ['solvers_count']

lines = []
lines.append(r"\begin{tabular}{l" + "r" * len(columns) + "}")
lines.append(r"\toprule")

header_cols = [latex_escape(df.index.name or "Index")] + [latex_escape(col) for col in columns]
lines.append(" & ".join(header_cols) + r" \\")
lines.append(r"\midrule")

for entry in combined_rows:
    if entry[0] == "__GROUP_HEADER__":
        prefix = latex_escape(entry[1])
        lines.append(r"\midrule")
        lines.append(r"\multicolumn{" + str(len(columns)+1) + r"}{l}{\textbf{Problem Class: " + prefix + r"}} \\")
        lines.append(r"\midrule")
    elif entry[0] == "__UNSOLVED__":
        unsolved_list = [latex_escape(x) for x in entry[1]]
        lines.append(f"% Unsolved problems in this class: {unsolved_list}")
    else:
        idx, row = entry
        idx_str = latex_escape(idx)
        row_vals = [latex_escape(row[col]) for col in columns]
        lines.append(" & ".join([idx_str] + row_vals) + r" \\")

lines.append(r"\bottomrule")
lines.append(r"\end{tabular}")

with open("./out/alive_solver_comparion.tex", "w") as f:
    f.write("\n".join(lines))
    f.write("\n")



=== Problem Class: AddSub ===
                 lemma  no_lemma    egraph  cvc4_tplanes  cvc4_tplanes_saturate_no_e_matching  vampire  z3_default  solvers_count
AddSub_1202        NaN       NaN  0.000405           NaN                                  NaN      NaN         NaN              1
AddSub_1564        NaN       NaN  0.001942           NaN                                  NaN      NaN         NaN              1
AddSub_1560     28.365       NaN  0.000055           NaN                                  NaN      NaN         NaN              2
AddSub_1624    283.636       NaN  0.000503           NaN                                  NaN      NaN         NaN              2
AddSub_1295    391.316       NaN  0.000661           NaN                                  NaN      NaN         NaN              2
AddSub_1574     27.106       NaN  0.000639          0.41                                12.73      NaN         NaN              4
AddSub_1619     27.162       NaN  0.000645          0.90   